In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import helpers as dsfh
import missingno as msno
from pyaxis import pyaxis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model
import statsmodels.api as sm #statsmodels is used for creating linear regression models
from sklearn.model_selection import train_test_split #creating testing and training data
from sklearn.metrics import roc_auc_score #evaluate model performance
import predhelp as ph
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures

In [138]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  24 non-null     int64  
 1   Herkunftsland - Total  24 non-null     float64
 2   pred                   24 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 704.0 bytes


In [70]:
df2 = pd.read_csv("../../dsfgp/data/cleaned data/total_df.csv").drop(columns=["Unnamed: 0"])

In [3]:
df = pd.read_csv("../../dsfgp/data/cleaned data/total_df.csv").drop(columns=["Unnamed: 0"])
df = df.loc[:, df.columns != 'DATE'].astype(float)

listen = ['Argentina',
 'Australien',
 'Austria',
 'Bahrain',
 'Belarus',
 'Belgium',
 'Brasil',
 'Bulgaria',
 'Canada',
 'China',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Egypt',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hongkong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Kuwait',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mexico',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Oman',
 'Philippinen',
 'Poland',
 'Portugal',
 'Qatar',
 'Romania',
 'Russia',
 'Saudi Arabia',
 'Serbia',
 'Singapore',
 'Slovakia',
 'Slovenia',
 'South Africa',
 'South Korea',
 'Spain',
 'Sweden',
 'Switzerland',
 'Taiwan',
 'Thailand',
 'Turkey',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 "Herkunftsland - Total"]

In [ ]:
y = df["Herkunftsland - Total"]
#X = df.drop(["Herkunftsland - Total"], axis=1)
X = df.drop(columns=listen)

In [141]:
X

,guests-St. Moritz,guests-Arosa,guests-Zermatt,AUS_GDP,AUT_GDP,BEL_GDP,BGR_GDP,CAN_GDP,CHE_GDP,CHL_GDP,...,cm neuschnee - gsg,cm avg. snowheight - gsg,mm rain - gsg,days rain >1mm - gsg,days rain >10mm - gsg,cm neuschnee - zwe,cm avg. snowheight - zwe,mm rain - zwe,days rain >1mm - zwe,days rain >10mm - zwe
0,96769.0,73314.0,141988.0,45427.60,47334.3,43042.50,16400.10,43545.0,60881.10,22025.8,...,76.0,57.0,97.1,13.0,4.0,33.0,5.0,66.8,9.0,2.0
1,92543.0,87284.0,155304.0,45427.60,47334.3,43042.50,16400.10,43545.0,60881.10,22025.8,...,120.0,86.0,138.0,13.0,2.0,119.0,27.0,105.3,11.0,3.0
2,91716.0,77321.0,177233.0,45427.60,47334.3,43042.50,16400.10,43545.0,60881.10,22025.8,...,63.0,78.0,76.8,11.0,2.0,42.0,10.0,69.2,13.0,3.0
3,18222.0,7010.0,82763.0,46507.80,47667.2,43530.70,16476.40,44044.2,61471.40,22179.4,...,21.0,29.0,120.0,11.0,4.0,8.0,0.0,92.9,9.0,4.0
4,17055.0,307.0,34342.0,46507.80,47667.2,43530.70,16476.40,44044.2,61471.40,22179.4,...,0.0,0.0,210.7,21.0,6.0,0.0,0.0,163.2,18.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,20174.0,816.0,47874.0,65018.00,64737.4,63151.30,29439.50,57252.1,82507.90,30855.3,...,0.0,0.0,98.7,9.0,4.0,0.0,0.0,73.2,6.0,3.0
113,54310.0,10644.0,88614.0,65018.00,64737.4,63151.30,29439.50,57252.1,82507.90,30855.3,...,0.0,0.0,126.2,16.0,5.0,0.0,0.0,109.2,12.0,4.0
114,101131.0,36962.0,154503.0,61211.34,61390.7,60394.92,28613.54,54148.2,78865.66,29569.5,...,0.0,0.0,75.7,7.0,3.0,0.0,0.0,47.6,6.0,1.0
115,107316.0,41335.0,146419.0,61211.34,61390.7,60394.92,28613.54,54148.2,78865.66,29569.5,...,0.0,0.0,84.6,9.0,5.0,0.0,0.0,92.5,7.0,4.0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [143]:
X = sm.add_constant(X_train)
olsres = sm.OLS(y_train,X_train)

In [144]:
model2 = olsres.fit()

In [145]:
print(model2.summary())

                              OLS Regression Results                             
Dep. Variable:     Herkunftsland - Total   R-squared:                       1.000
Model:                               OLS   Adj. R-squared:                    nan
Method:                    Least Squares   F-statistic:                       nan
Date:                   Fri, 02 Dec 2022   Prob (F-statistic):                nan
Time:                           00:47:09   Log-Likelihood:                 1631.7
No. Observations:                     93   AIC:                            -3077.
Df Residuals:                          0   BIC:                            -2842.
Df Model:                             92                                         
Covariance Type:               nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

/usr/local/Caskroom/miniconda/base/envs/dsf/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/Caskroom/miniconda/base/envs/dsf/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/Caskroom/miniconda/base/envs/dsf/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [146]:
#interpret the correlations in the dataframe
df.corr()

,Argentina,Australien,Austria,Bahrain,Belarus,Belgium,Brasil,Bulgaria,Canada,China,...,cm neuschnee - gsg,cm avg. snowheight - gsg,mm rain - gsg,days rain >1mm - gsg,days rain >10mm - gsg,cm neuschnee - zwe,cm avg. snowheight - zwe,mm rain - zwe,days rain >1mm - zwe,days rain >10mm - zwe
Argentina,1.000000,0.438428,0.455130,0.276178,0.285083,0.357407,0.352341,0.182852,0.441103,0.631199,...,-0.121718,-0.013449,0.092898,0.072565,0.145398,0.016888,0.020205,0.139620,0.084860,0.150896
Australien,0.438428,1.000000,0.504140,0.251377,0.160082,0.416508,0.632224,0.352375,0.701872,0.379020,...,0.132608,0.127336,-0.002239,-0.006547,0.034788,0.044340,0.011207,0.069651,0.004224,0.050034
Austria,0.455130,0.504140,1.000000,0.206434,0.077852,0.343236,0.488048,0.244677,0.575960,0.508148,...,-0.066980,-0.029605,0.162913,0.108083,0.192633,0.105960,0.173041,0.234190,0.157184,0.235598
Bahrain,0.276178,0.251377,0.206434,1.000000,0.084727,0.154594,0.216866,0.042597,0.181808,0.133414,...,0.015250,-0.052356,-0.085012,-0.127256,-0.023721,0.073597,-0.015720,-0.070815,-0.110085,-0.024627
Belarus,0.285083,0.160082,0.077852,0.084727,1.000000,0.046190,0.037225,-0.006040,0.054964,0.120891,...,-0.047825,-0.028249,-0.023871,0.029471,-0.043125,0.264760,0.112976,0.029195,0.037667,-0.014771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cm neuschnee - zwe,0.016888,0.044340,0.105960,0.073597,0.264760,0.394049,0.210409,0.317746,0.009231,-0.053288,...,0.134244,0.290609,-0.038014,0.004455,-0.076573,1.000000,0.842106,-0.038074,0.022023,-0.058214
cm avg. snowheight - zwe,0.020205,0.011207,0.173041,-0.015720,0.112976,0.470394,0.235841,0.396793,0.054461,-0.033741,...,0.061792,0.224928,-0.069240,-0.059303,-0.101734,0.842106,1.000000,-0.072824,-0.030736,-0.086230
mm rain - zwe,0.139620,0.069651,0.234190,-0.070815,0.029195,-0.012187,0.016694,0.003105,0.080657,0.083627,...,0.009180,-0.144967,0.898602,0.752165,0.836350,-0.038074,-0.072824,1.000000,0.819128,0.921069
days rain >1mm - zwe,0.084860,0.004224,0.157184,-0.110085,0.037667,0.011553,-0.032456,0.020938,-0.002970,-0.005272,...,0.124325,-0.052989,0.802124,0.910774,0.766966,0.022023,-0.030736,0.819128,1.000000,0.728979


In [147]:
ypred = olsres.predict(X_train)

ValueError: shapes (93,164) and (93,164) not aligned: 164 (dim 1) != 93 (dim 0)

In [148]:
ols1 = LinearRegression() 

In [149]:
ols1.fit(X_train, y_train)
crops = ols1.predict(X_train)

# Get value of constant and coefficient
print(f"constant: {ols1.intercept_:.2f}, coefficient: {ols1.coef_[0]:.7f}.")

constant: -465340.78, coefficient: 0.2901861.


In [150]:
mse = np.mean((crops - y_train) ** 2)
ols1.score(X_train, y_train)

1.0

In [151]:
crops2 = ols1.predict(X_test)
print(f"constant: {ols1.intercept_:.2f}, coefficient: {ols1.coef_[0]:.7f}.")


constant: -465340.78, coefficient: 0.2901861.


In [152]:
mse = np.mean((crops2 - y_test) ** 2)
ols1.score(X_test, y_test)


0.28511501427106667

In [153]:
mae3 = np.mean(np.abs(crops2 - y_test))
mae3 # Display the MAE

10022.643199796497

In [154]:
df = y_test
#df["true"] = pd.DataFrame(crops2)

In [155]:
df = df.reset_index()
df["pred"] = pd.DataFrame(crops2)

In [7]:
# 2 degrees of polynomials, without the constant
poly2 = PolynomialFeatures(2, include_bias=False) 
# Define a new X with the squared feature
X2 = poly2.fit_transform(X_train)
X2[:5, :] # Check the first five rows of our new features
ols2 = LinearRegression()

In [167]:
# Fit the estimator 
ols2.fit(X2, y_train)
# Add the predictions to our `crops` dataframe
X2_test = poly2.fit_transform(X_test)
cropsy = ols2.predict(X2_test)


In [56]:
df2['Date'] = pd.to_datetime(df2.DATE, format='%Y-%m-%d')



In [71]:
df2['season'] = (df2['Date'].dt.month%12 + 3)//3

seasons = {
             1: 'Winter',
             2: 'Spring',
             3: 'Summer',
             4: 'Autumn'
}

df2['season_name'] = df2['season'].map(seasons)

KeyError: 'Date'

In [59]:
w = df2[df2["season"] == 1].reset_index().drop(columns=["index","DATE","Date","season","season_name"])
f = df2[df2["season"] == 2].reset_index().drop(columns="index")
s = df2[df2["season"] == 3].reset_index().drop(columns="index")
h = df2[df2["season"] == 4].reset_index().drop(columns="index")

In [72]:
def split_seasons(df):
    df['Date'] = pd.to_datetime(df.DATE, format='%Y-%m-%d')
    df['season'] = (df['Date'].dt.month%12 + 3)//3
    df['season_name'] = df['season'].map(seasons)

    w = df[df["season"] == 1].reset_index().drop(columns=["index","DATE","Date","season","season_name"])
    f = df[df["season"] == 2].reset_index().drop(columns=["index","DATE","Date","season","season_name"])
    s = df[df["season"] == 3].reset_index().drop(columns=["index","DATE","Date","season","season_name"])
    h = df[df["season"] == 4].reset_index().drop(columns=["index","DATE","Date","season","season_name"])
    return(w,f,s,h)

In [73]:
w,f,s,h = split_seasons(df2)

In [74]:
w

,Argentina,Australien,Austria,Bahrain,Belarus,Belgium,Brasil,Bulgaria,Canada,China,...,cm neuschnee - gsg,cm avg. snowheight - gsg,mm rain - gsg,days rain >1mm - gsg,days rain >10mm - gsg,cm neuschnee - zwe,cm avg. snowheight - zwe,mm rain - zwe,days rain >1mm - zwe,days rain >10mm - zwe
0,8.0,218.0,98.0,10.0,32.0,721.0,331.0,9.0,159.0,29.0,...,76,57,97.1,13,4,33,5,66.8,9,2
1,12.0,13.0,180.0,21.0,2.0,2023.0,323.0,176.0,82.0,84.0,...,120,86,138.0,13,2,119,27,105.3,11,3
2,0.0,68.0,125.0,0.0,0.0,361.0,69.0,25.0,77.0,241.0,...,0,19,59.7,7,2,17,7,42.1,4,1
3,53.0,267.0,191.0,6.0,630.0,658.0,394.0,54.0,120.0,65.0,...,0,40,87.5,9,5,47,9,83.6,11,3
4,62.0,80.0,183.0,2.0,0.0,846.0,235.0,242.0,66.0,220.0,...,0,80,152.7,18,6,48,13,105.2,11,6
5,6.0,42.0,82.0,132.0,593.0,449.0,107.0,34.0,52.0,168.0,...,64,6,94.1,12,3,56,4,76.4,8,2
6,95.0,170.0,102.0,8.0,447.0,685.0,180.0,17.0,97.0,63.0,...,0,22,162.0,17,8,81,12,119.8,18,4
7,36.0,46.0,202.0,0.0,0.0,1739.0,337.0,104.0,127.0,91.0,...,0,0,66.0,6,3,50,33,48.3,7,1
8,0.0,70.0,126.0,0.0,0.0,374.0,45.0,19.0,80.0,157.0,...,0,16,18.8,6,0,0,1,15.8,5,0
9,23.0,146.0,48.0,10.0,40.0,637.0,146.0,56.0,70.0,102.0,...,122,47,257.1,21,8,0,0,139.6,17,5


In [78]:
y2 = w["Herkunftsland - Total"]
#X = df.drop(["Herkunftsland - Total"], axis=1)
X2 = w.drop(columns=listen)

In [81]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.20)

In [85]:
# 2 degrees of polynomials, without the constant
poly2 = PolynomialFeatures(2, include_bias=False) 
# Define a new X with the squared feature
X3 = poly2.fit_transform(X2_train)
X3[:5, :] # Check the first five rows of our new features
ols2 = LinearRegression()

In [86]:
# Fit the estimator 
ols2.fit(X3, y2_train)
# Add the predictions to our `crops` dataframe
X2_test = poly2.fit_transform(X2_test)
cropsy = ols2.predict(X2_test)


In [88]:
mae3 = np.mean(np.abs(cropsy - y2_test))
mae3 # Display the MAE

2828.821778881878

In [96]:
prediction = pd.DataFrame(y2_test).reset_index()
prediction["pred"] = cropsy

In [97]:
prediction

,index,Herkunftsland - Total,pred
0,13,49478.0,49442.172684
1,11,27217.0,31252.191250
2,0,39240.0,39672.814652
3,22,59058.0,49985.712725
4,7,48506.0,47044.627753
5,1,46474.0,48409.437933


In [94]:
y2_test

13    49478.0
11    27217.0
0     39240.0
22    59058.0
7     48506.0
1     46474.0
Name: Herkunftsland - Total, dtype: float64